In [19]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import matplotlib.pyplot as plt

In [21]:
df = pd.read_json('jsons/result1.json')

df = json_normalize(df['messages'])
channel_name = df['from'].dropna().iloc[0]
df = df[['date', 'text_entities']]

In [22]:
df.head()

,date,text_entities
0,2021-07-17T02:04:08,[]
1,2021-07-17T02:30:57,"[{'type': 'bold', 'text': 'Всем привет!'}, {'t..."
2,2021-07-17T02:46:12,"[{'type': 'bold', 'text': 'Перед размещением о..."
3,2021-08-18T22:32:41,"[{'type': 'hashtag', 'text': '#Ищу'}, {'type':..."
4,2021-08-18T22:35:34,"[{'type': 'hashtag', 'text': '#АлмаАта'}, {'ty..."


In [25]:
df = df[df['text_entities'].apply(lambda x: isinstance(x, list) and len(x) > 0)]

def extract_message(text_entities):
    try:
        # Concatenate the 'text' fields from each dictionary in the list
        return ''.join([entity['text'] for entity in text_entities if 'text' in entity])
    except:
        return None

# Apply the function to create a new 'message' column
df['message'] = df['text_entities'].apply(extract_message)

# Drop the 'text_entities' column as it is no longer needed
df_cleaned = df.drop(columns=['text_entities'])
df_cleaned = df_cleaned.sort_values(by='date', ascending=False)
df_cleaned

,date,message
45859,2024-10-27T18:19:25,"Ищем одного парня на подселение, будем жить 4о..."
45858,2024-10-27T17:44:07,СДУ ға өте жақын жерде орналасқан\nКотеждің бі...
45857,2024-10-27T17:07:36,Ноябрь айынан бастап бірге тұруға 1қыз іздеймі...
45856,2024-10-27T17:05:52,"Сәлеметсіздер ме, 2қыз бала керек на подселени..."
45855,2024-10-27T16:56:46,Всем привет! Ищу одну девушку на подселение. 2...
...,...,...
5,2021-08-18T23:27:20,#ищу #Каскелен #квартира #3_комнаты\nАрендуем ...
4,2021-08-18T22:35:34,#АлмаАта #Подселение #Квартира\nИщем девушку и...
3,2021-08-18T22:32:41,#Ищу #Подселение #АлмаАта #Квартира #Дом #2_ко...
2,2021-07-17T02:46:12,Перед размещением объявления рекомендуем испол...


In [27]:
# df_cleaned = df_cleaned[df_cleaned['date'] >= '2024-10-10']
df_cleaned = df_cleaned.head(50)
len(df_cleaned)

500

In [29]:
# Helper function to safely extract values
def extract_value(pattern, text, data_type=str):
    match = re.search(pattern, text)
    if match:
        value = match.group(1).strip()
        if data_type == int:
            try:
                return int(value)
            except ValueError:
                return None
        elif data_type == bool:
            return value.lower() == 'true'
        return value
    return None

In [ ]:
import json
from gemini2 import gemini
from parse import parse_string
import re
import time
import numpy as np

# Initialize lists to store the extracted values for each column
cities = []
districts = []
addresses = []
costs = []
genders = []
people_neededs = []
long_terms = []
phone_numbers = []
utilities = []
deposits = []

total_rows = len(df_cleaned)

# Iterate over the first 10 rows of df_cleaned
for start in range(0, total_rows, 10):
    end = min(start + 20, total_rows)  # Ensure we don't go out of bounds
    
    for i in range(len(df_cleaned)):
        message = df_cleaned['message'].iloc[i].replace('\n', '    ') + " Алматы"
        result_text = gemini(message)  # Process the message with the gemini function
    
        # Extract each field using extract_value
        city = extract_value(r'city:\s*(.*)', result_text)
        district = extract_value(r'district:\s*(.*)', result_text)
        address = extract_value(r'address:\s*(.*)', result_text)
        cost = extract_value(r'cost:\s*(\d+)', result_text, int)
        gender = extract_value(r'gender:\s*(.*)', result_text)
        people_needed = extract_value(r'people_needed:\s*(\d+)', result_text, int)
        long_term = extract_value(r'long_term:\s*(.*)', result_text, bool)
        phone_number = extract_value(r'phone_number:\s*(.*)', result_text)
        utility = extract_value(r'utilities:\s*(.*)', result_text, bool)
        deposit = extract_value(r'deposit:\s*(\d+)', result_text, int)
    
        # Append the extracted values to the lists
        cities.append(city)
        districts.append(district)
        addresses.append(address)
        costs.append(float(cost) if cost is not None else np.nan)  # Convert to float or NaN
        genders.append(gender)
        people_neededs.append(float(people_needed) if people_needed is not None else np.nan)  # Convert to float or NaN
        long_terms.append(long_term)
        phone_numbers.append(phone_number)
        utilities.append(utility)
        deposits.append(float(deposit) if deposit is not None else np.nan)  # Convert to float or NaN
            
    time.sleep(5)

df_cleaned['city'] = cities
df_cleaned['district'] = districts
df_cleaned['address'] = addresses
df_cleaned['cost'] = costs
df_cleaned['gender'] = genders
df_cleaned['people_needed'] = people_needed
df_cleaned['long_term'] = long_terms
df_cleaned['phone_number'] = phone_numbers
df_cleaned['utilities'] = utilities
df_cleaned['deposit'] = deposits

# Display the updated DataFrame
df_cleaned

In [17]:
df_cleaned.to_excel('res.xlsx')

In [ ]:
import time
import json
from gemini1 import gemini
from parse import parse_string
import pandas as pd
import re

# Initialize the total number of rows
total_rows = len(df_cleaned)

# Define a function to process each message with retries for rate limiting
def process_message_with_retry(message, retries=3):
    for attempt in range(retries):
        try:
            result = gemini(message)
            if result:
                return parse_string(result)
            else:
                return None
        except Exception as e:
            if "429" in str(e):
                wait_time = (2 ** attempt) * 10  # Exponential backoff
                print(f"Rate limit exceeded, retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                print(f"Error processing message: {e}")
                return None
    return None

# Load existing data if available
try:
    df_cleaned = pd.read_excel("df_cleaned_progress_temp.xlsx")
except FileNotFoundError:
    pass  # If there's no file, we'll start fresh

# Process data in chunks of 15 rows and save progress after each chunk
for start in range(0, total_rows, 15):
    end = min(start + 15, total_rows)  # Ensure we don't go out of bounds
    
    # Temporary lists to store values for this chunk
    cities = []
    districts = []
    addresses = []
    costs = []
    genders = []
    people_needed = []
    long_terms = []
    phone_numbers = []
    utilities = []
    deposits = []

    for i in range(start, end):
        try:
            message = df_cleaned['message'].iloc[i] + " " + channel_name
            structured_data = process_message_with_retry(message)  # Process message with retry logic

            # Append parsed data or None if result is invalid
            cities.append(structured_data.get('city') if structured_data else None)
            districts.append(structured_data.get('district') if structured_data else None)
            addresses.append(None)  # Placeholder for address
            costs.append(structured_data.get('cost') if structured_data else None)
            genders.append(structured_data.get('gender') if structured_data else None)
            people_needed.append(structured_data.get('people_needed') if structured_data else None)
            long_terms.append(structured_data.get('long_term') if structured_data else None)
            phone_numbers.append(structured_data.get('phone_number') if structured_data else None)
            utilities.append(structured_data.get('utilities') if structured_data else None)
            deposits.append(structured_data.get('deposit') if structured_data else None)
                
        except Exception as e:
            print(f"Error processing row {i}: {e}")
            cities.append(None)
            districts.append(None)
            addresses.append(None)
            costs.append(None)
            genders.append(None)
            people_needed.append(None)
            long_terms.append(None)
            phone_numbers.append(None)
            utilities.append(None)
            deposits.append(None)

    # After each chunk, create a DataFrame for the chunk
    chunk_df = pd.DataFrame({
        'city': cities,
        'district': districts,
        'address': addresses,
        'cost': costs,
        'gender': genders,
        'people_needed': people_needed,
        'long_term': long_terms,
        'phone_number': phone_numbers,
        'utilities': utilities,
        'deposit': deposits
    }).dropna(how='all')  # Drop rows if all values are NaN

    # Append chunk to df_cleaned only if chunk_df is not empty
    if not chunk_df.empty:
        df_cleaned = pd.concat([df_cleaned, chunk_df], ignore_index=True)

    # Save progress to a temporary file after each chunk
    df_cleaned.to_excel("df_cleaned_progress_temp.xlsx", index=False)

    # Wait for 60 seconds if rate limit errors are frequent
    time.sleep(60)

# Final save after processing all chunks
df_cleaned.to_excel("df_cleaned_final.xlsx", index=False)


In [ ]:
df_cleaned.to_excel("df_cleaned_final.xlsx")

date                                  2024-10-27T16:23:45
message    Подселениеге 2жігіт керек.Ауезов ауданында үй.
Name: 45844, dtype: object